In [1]:
import sys
sys.path.append("..")

from libs.InteractivePitchUI import *
from libs.data_manipulation import *
from libs.dim_reduction import *
from libs.feature_generation import *
from libs.clustering import *
from libs.convex_hull import *
from libs.alpha_shape import *
from libs.weight_generator import *
from libs.similar_movement import *
from libs.Video_Player import *

/home/martin/.local/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (None)/charset_normalizer (2.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
matches = compile_team_tracking_data_with_labels("../data", "Denmark", "../data/Labelled_ground_truths.csv")




Index(['Time [s]_team', 'half_team', 'minute_team', 'Period_team', 'home_6_x',
       'home_6_y', 'home_18_x', 'home_18_y', 'home_1_x', 'home_1_y',
       'home_10_x', 'home_10_y', 'home_3_x', 'home_3_y', 'home_2_x',
       'home_2_y', 'home_19_x', 'home_19_y', 'home_21_x', 'home_21_y',
       'home_9_x', 'home_9_y', 'home_17_x', 'home_17_y', 'home_23_x',
       'home_23_y', 'home_20_x', 'home_20_y', 'home_15_x', 'home_15_y',
       'home_8_x', 'home_8_y', 'home_5_x', 'home_5_y', 'home_12_x',
       'home_12_y', 'ball_z_team', 'Time [s]', 'half', 'minute', 'Period',
       'away_10_x', 'away_10_y', 'away_2_x', 'away_2_y', 'away_17_x',
       'away_17_y', 'away_22_x', 'away_22_y', 'away_9_x', 'away_9_y',
       'away_1_x', 'away_1_y', 'away_20_x', 'away_20_y', 'away_21_x',
       'away_21_y', 'away_11_x', 'away_11_y', 'away_13_x', 'away_13_y',
       'away_6_x', 'away_6_y', 'away_7_x', 'away_7_y', 'away_5_x', 'away_5_y',
       'away_14_x', 'away_14_y', 'away_23_x', 'away_23_y', 'away_1

In [3]:
df = extract_one_match(matches,4)

In [4]:
df = df[df["match_name"] == "Denmark_England"]




In [5]:
func_control = lambda x:1
func_inverse = lambda x: 20/x
func_linear = lambda x: 200-x
func_exp = lambda x: np.exp(-x/20)

In [6]:
df.Label.unique()

array(['Missing', 'run-into-space', 'breakthrough-wb',
       'breakthrough-space', 'breakthrough-even'], dtype=object)

In [7]:
import datetime


def process_situations(df):
    # Dictionary to store recommendations
    recommendations_dict = {}

    # Step 2: Extract relevant rows (all rows with a label not equal to "Missing")
    labeled_rows = df[df['Label'] != "Missing"]
    relevant_indices = labeled_rows.index.tolist()
    print(relevant_indices)
    # Step 2: Process each relevant index
    # Step 2: Process each relevant index
    for relevant_index in relevant_indices:
        # Step 3: Apply Wasserstein distance for various functions upfront
        finished_indices_control = most_similar_with_wasserstein(relevant_index, df, func_control)
        finished_indices_linear = most_similar_with_wasserstein(relevant_index, df, func_linear)
        finished_indices_inverse = most_similar_with_wasserstein(relevant_index, df, func_inverse)
        finished_indices_exp = most_similar_with_wasserstein(relevant_index, df, func_exp)

        # Convert to numpy arrays
        finished_indices_control = np.array(finished_indices_control)
        finished_indices_exp = np.array(finished_indices_exp)
        finished_indices_linear = np.array(finished_indices_linear)
        finished_indices_inverse = np.array(finished_indices_inverse)

        # Step 4: Filter indices based on the time window (±480 seconds)
        time_window_filter = lambda indices: indices[
            (indices > relevant_index + 480) | 
            (indices < relevant_index - 480)
        ]

        finished_indices_control = time_window_filter(finished_indices_control)[:100]
        finished_indices_exp = time_window_filter(finished_indices_exp)[:100]
        finished_indices_linear = time_window_filter(finished_indices_linear)[:100]
        finished_indices_inverse = time_window_filter(finished_indices_inverse)[:100]

        # Combine recommendations into a dictionary for separate processing
        wasserstein_recommendations = {
            "Control": finished_indices_control,
            "Exp": finished_indices_exp,
            "Linear": finished_indices_linear,
            "Inverse": finished_indices_inverse
        }
        # Create the folder based on the label
        # Step 6: Prepare the result DataFrame for this function set
        label = df.loc[relevant_index, 'Label']  # Get the label for the situation
        match_name = df.loc[relevant_index, 'match_name']
        time_of_situation = str(datetime.timedelta(seconds=df.loc[relevant_index, 'Time [s]']))

        folder_name = f"{label}_{match_name}_{time_of_situation}"
        generate_folder(folder_name)

        # Save the time and match_name to a text file
        with open(f"{folder_name}/situation_info.txt", "w") as f:
                f.write(f"Match Name: {match_name}\n")
                f.write(f"Time of Situation: {time_of_situation}\n")

        # Step 5: Process each set of indices separately
        for func_name, indices_set in wasserstein_recommendations.items():
            print(indices_set, func_name)
            if len(indices_set) == 0:
                continue  # Skip if no indices in this set

            # Step 5.1: Compute similar movements for the current set
            similar_movement = find_similar_movement(df, relevant_index, indices_set,length_of_sequence=240)
            similar_movement_ranked = sorted(similar_movement, key=lambda x: x[0])

            # Step 5.2: Select indices, ensuring no two are within 100 places of each other
            selected_indices = []
            for _, index in similar_movement_ranked:
                if all(abs(index - selected) > 500 for selected in selected_indices):
                    selected_indices.append(index)
                if len(selected_indices) == 10:  # Stop once we have 10 indices
                    break

            new_df = df.loc[selected_indices[:10]]

            
            


            # Generate the result DataFrame
            result_df = new_df[["match_name", "Time [s]", "half_team"]]
            result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))

            # Save the ground truth CSV for this function set
            result_df.to_csv(f"{folder_name}/{folder_name}_{func_name}_ground_truth.csv")

            # Shuffle and save the ordering truth CSV for this function set
            result_df.sample(frac=1).to_csv(f"{folder_name}/{folder_name}_{func_name}_for_ordering_truth.csv")

            # Store the recommendations in the dictionary
            recommendations_dict[relevant_index] = recommendations_dict.get(relevant_index, {})
            recommendations_dict[relevant_index][func_name] = selected_indices[:10]

    # Return the dictionary with recommendations
    return recommendations_dict

    

reccomendations_dict = process_situations(df)

[143547, 154347, 181797, 189572, 199447, 208722, 214138]
Folder 'run-into-space_Denmark_England_0:00:10' does not exist.
[169697 169649 169745 167585 167537 266753 266801 169601 241697 242129
 252449 169793 232529 161825 161777 203681 241649 226673 220385 167489
 203729 281105 183905 277265 284273 277313 195713 167633 203777 228401
 169553 266849 232481 284369 266705 203825 284417 162545 221153 284321
 220337 252497 282497 212177 232577 232673 241601 281921 281969 234449
 232625 282017 220001 161729 212225 242417 169841 183377 195761 221105
 281057 281873 234209 284465 195665 183329 282065 220289 284225 183425
 162593 212129 234257 242369 232721 276881 281825 220625 282737 234401
 251441 206369 161873 282353 276833 183857 277217 242465 276929 162161
 206465 228449 277121 242177 284657 277073 276977 282113 277025 203873] Control


/tmp/ipykernel_28056/1268882290.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))


[266609 281057 261521 252401 241697 203825 252449 203921 220577 218753
 203873 169889 261233 169937 266657 153473 167489 261473 167345 160289
 167537 223889 266705 183905 268865 266561 164849 220241 162161 266753
 211745 250289 164801 169985 172145 180689 282353 167393 167201 220193
 162305 241649 211697 250241 203969 172193 164513 219953 167441 282977
 226625 201905 212177 284225 234113 220625 201617 203633 220961 261425
 183857 170033 261377 205505 183377 183425 273089 218801 183329 183569
 183473 161249 183521 183617 183281 160241 183233 261329 162113 183665
 212369 183185 182753 161777 183713 236129 180545 211793 211217 282449
 183137 195713 164465 183761 205409 183809 222929 230321 268913 205457] Exp


/tmp/ipykernel_28056/1268882290.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))


[169697 169649 169745 167585 167537 266753 266801 169601 241697 242129
 252449 169793 232529 161825 161777 203681 241649 226673 220385 167489
 203729 281105 183905 277265 284273 277313 195713 167633 203777 228401
 169553 266849 232481 284369 266705 203825 284417 162545 221153 284321
 220337 252497 282497 212177 232577 232673 241601 281921 281969 234449
 232625 282017 220001 161729 212225 242417 169841 183377 195761 221105
 281057 281873 234209 284465 195665 183329 282065 220289 284225 183425
 162593 212129 234257 242369 232721 276881 281825 220625 282737 234401
 251441 206369 161873 282353 276833 183857 277217 242465 276929 162161
 206465 228449 277121 242177 284657 277073 276977 282113 277025 203873] Linear


...

Folder 'breakthrough-even_Denmark_England_0:46:06' does not exist.
[215105 215057 158225 215153 197777 166865 212753 176129 250001 249857
 166913 202049 158177 166961 243521 204737 215009 271793 171425 203489
 204689 221681 201617 271745 158129 221729 222113 259985 197825 186497
 215201 249953 151937 264833 167009 224465 212705 176081 210257 237521
 166577 265025 237473 172577 260033 221777 145793 210305 264401 224513
 160913 250049 272945 166481 210449 260129 212609 278465 186449 259889
 213233 158081 260081 203441 210401 224417 166529 259937 151985 271697
 210353 243569 271841 222065 210209 222161 237905 222737 144737 212657
 157745 249905 167057 166817 221825 157697 283649 221873 168641 215249
 237569 168929 186545 150737 237425 197729 224321 166433 264449 214961] Control


/tmp/ipykernel_28056/1268882290.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))


[275249 257393 218993 237953 210833 215441 198545 238001 262481 262433
 234017 204977 210977 215681 274577 260033 145841 215825 219041 281633
 274625 188897 173729 180497 210929 151169 278129 184961 159041 164657
 274673 221873 143585 197537 198497 205217 219137 167105 158369 144977
 238049 265265 211121 221681 205169 219089 274721 222833 281729 281681
 276209 224465 169073 169025 262385 215345 182321 259985 210881 276161
 167057 198593 164081 197585 237905 274529 151121 211025 198449 238193
 263585 215633 260177 144689 274769 281777 217169 173825 264881 276401
 165329 231137 147953 261905 171713 152417 257825 152753 250145 272321
 219425 231185 161057 272129 218897 161009 215873 210737 152609 147857] Exp
[215105 215057 158225 215153 197777 166865 212753 176129 250001 249857
 166913 202049 158177 166961 243521 204737 215009 271793 171425 203489
 204689 221681 201617 271745 158129 221729 222113 259985 197825 186497
 215201 249953 151937 264833 167009 224465 212705 176081 210257 237521
 

/tmp/ipykernel_28056/1268882290.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))
/tmp/ipykernel_28056/1268882290.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))
/tmp/ipykernel_28056/1268882290.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

[257393 275249 260033 221681 197537 237953 224465 264929 151169 221873
 145841 152417 259985 260081 167057 232337 237905 167105 212609 161009
 264977 262001 180497 218993 219137 182321 212849 186641 186593 151121
 166913 204977 250049 264881 272753 144737 219041 176081 272417 219185
 238001 144977 272705 184961 173729 219089 222737 188897 215393 188657
 219425 265025 204929 261953 147377 158225 145361 219857 221633 167153
 160961 169025 186689 145793 197489 198545 215873 261857 186545 182417
 280241 176033 153761 224417 222881 215441 260129 215825 261137 151217
 168929 169073 198497 168977 151073 238049 222833 218897 166625 209441
 144689 158273 159041 262385 265073 224513 221921 260177 264833 182369] Inverse


In [8]:
reccomendations_dict

{143547: {'Control': [234257,
   252497,
   282353,
   169793,
   241601,
   203681,
   284465,
   281105,
   167633,
   232577],
  'Exp': [282977,
   261521,
   230321,
   268913,
   164465,
   282353,
   218753,
   160241,
   284225,
   180689],
  'Linear': [234257,
   252497,
   282353,
   169793,
   241601,
   203681,
   284465,
   281105,
   167633,
   232577],
  'Inverse': [282353,
   234401,
   169793,
   241601,
   203681,
   261329,
   284225,
   281057,
   252449,
   218273]},
 154347: {'Control': [189521,
   173825,
   187841,
   198641,
   278705,
   175697,
   214289,
   187025,
   231233,
   171713],
  'Exp': [187745,
   186881,
   189473,
   173777,
   278657,
   185009,
   259169,
   151601,
   272753,
   175745],
  'Linear': [189521,
   173825,
   187841,
   198641,
   278705,
   175697,
   214289,
   187025,
   231233,
   171713],
  'Inverse': [173825,
   187841,
   278705,
   175697,
   186929,
   189473,
   151553,
   185009,
   231233,
   214241]},
 181797: {'Contr

In [10]:

root ="csvs_for_ranking"
folders = os.listdir(root)

import re

def extract_seconds_from_folder(folder_name):
    # Use regex to find the time pattern (hh:mm:ss)
    match = re.search(r'(\d+):(\d+):(\d+)', folder_name)
    if match:
        hours, minutes, seconds = map(int, match.groups())
        # Convert to total seconds
        return hours * 3600 + minutes * 60 + seconds
    else:
        # Return None if no time is found
        return None
2700


for folder in folders:
    csvs = os.listdir(root + "/"+folder)
    for csv in csvs:
        if ("ground_truth" in csv):
            print(folder)


            seconds = extract_seconds_from_folder(folder)

            if (seconds > 2700):
                myplayer = VideoPlayer("3-2024-2036178-Denmark_England.mp4",8+971+seconds,0,[])
            if (seconds < 2700):
                myplayer = VideoPlayer("3-2024-2036178-Denmark_England.mp4",8+seconds,0,[])
            df = pd.read_csv(root +"/"+folder+"/"+csv)
            indices_1 = df[["Time [s]","half_team",'Unnamed: 0']].sample(frac=1).to_numpy()
            myplayer = VideoPlayer("3-2024-2036178-Denmark_England.mp4",8,8+971,indices_1)
            ratings = myplayer.ratings
            ratings_df  = pd.DataFrame(ratings, columns=['Unnamed: 0',"rating"])
            rated_situations = pd.merge(df, ratings_df, how ="inner", on=['Unnamed: 0'])
            rated_situations =rated_situations.drop_duplicates('Unnamed: 0')
            rated_situations.to_csv(root +"/"+folder+"/"+"ranked_"+csv)




breakthrough-space_Denmark_England_0:37:26
Creating Pause Button
Creating Play Button
Creating Next Button
Creating Previous Button
Creating Close Button
Creating Time Label
Creating Negative Rating Button
Creating Positiv Rating Button


[00007f1b1c004ea0] gl gl: Initialized libplacebo v4.192.1 (API v192)


Initial Time: 0 seconds
Creating Pause Button
Creating Play Button
Creating Next Button
Creating Previous Button
Creating Close Button
Creating Time Label
Creating Negative Rating Button
Creating Positiv Rating Button


[00007f1af4001720] gl gl: Initialized libplacebo v4.192.1 (API v192)


Initial Time: 0 seconds


...

invalid command name "139756208969664update_time_label"
    while executing
"139756208969664update_time_label"
    ("after" script)


run-into-space_Denmark_England_0:00:10
Creating Pause Button
Creating Play Button
Creating Next Button
Creating Previous Button
Creating Close Button
Creating Time Label
Creating Negative Rating Button
Creating Positiv Rating Button


[00007f1a58001720] gl gl: Initialized libplacebo v4.192.1 (API v192)


Initial Time: 0 seconds


invalid command name "139756208975488update_time_label"
    while executing
"139756208975488update_time_label"
    ("after" script)
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/martin/anaconda3/envs/DBU/lib/python3.10/tkinter/__init__.py", line 1919, in __call__
    if self.subst:
KeyboardInterrupt


In [28]:
ratings_df  = pd.DataFrame(ratings, columns=['Unnamed: 0',"rating"])

In [33]:
rated_situations = pd.merge(df, ratings_df, how ="inner", on=['Unnamed: 0'])

In [34]:
rated_situations =rated_situations.drop_duplicates('Unnamed: 0')

In [35]:
print(rated_situations)

    Unnamed: 0       match_name  Time [s] half_team        Time Min  rating
0       179537  Denmark_England   1449.60        1H  0:24:09.600000       1
1       174689  Denmark_England   1255.68        1H  0:20:55.680000       0
3       199505  Denmark_England   2248.32        1H  0:37:28.320000       1
5       149921  Denmark_England    264.96        1H  0:04:24.960000       1
6       193697  Denmark_England   2016.00        1H         0:33:36       0
7       208001  Denmark_England   2588.16        1H  0:43:08.160000       1
8       238961  Denmark_England   3758.92        2H  1:02:38.920000       0
9       253697  Denmark_England   4348.36        2H  1:12:28.360000       0
10      176321  Denmark_England   1320.96        1H  0:22:00.960000       1
11      185441  Denmark_England   1685.76        1H  0:28:05.760000       1
